In [1]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ctransformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached torch-2.1.1-cp39-cp39-manylinux1_x86_64.whl (670.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.6 MB/s eta 0:00:00
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [5]:
pip install faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [2]:
DB_FAISS_PATH = "vectorstore/db_faiss"
loader = CSVLoader(file_path="data/2019.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
print(data)

[Document(page_content='Overall rank: 1\nCountry or region: Finland\nScore: 7.769\nGDP per capita: 1.340\nSocial support: 1.587\nHealthy life expectancy: 0.986\nFreedom to make life choices: 0.596\nGenerosity: 0.153\nPerceptions of corruption: 0.393', metadata={'source': 'data/2019.csv', 'row': 0}), Document(page_content='Overall rank: 2\nCountry or region: Denmark\nScore: 7.600\nGDP per capita: 1.383\nSocial support: 1.573\nHealthy life expectancy: 0.996\nFreedom to make life choices: 0.592\nGenerosity: 0.252\nPerceptions of corruption: 0.410', metadata={'source': 'data/2019.csv', 'row': 1}), Document(page_content='Overall rank: 3\nCountry or region: Norway\nScore: 7.554\nGDP per capita: 1.488\nSocial support: 1.582\nHealthy life expectancy: 1.028\nFreedom to make life choices: 0.603\nGenerosity: 0.271\nPerceptions of corruption: 0.341', metadata={'source': 'data/2019.csv', 'row': 2}), Document(page_content='Overall rank: 4\nCountry or region: Iceland\nScore: 7.494\nGDP per capita: 1.

In [3]:
# Split the text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [4]:
print(len(text_chunks))

156


In [5]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
# COnverting the text Chunks into embeddings and saving the embeddings into FAISS Knowledge Base
docsearch = FAISS.from_documents(text_chunks, embeddings)

In [7]:
docsearch.save_local(DB_FAISS_PATH)

In [8]:
llm = CTransformers(model="models/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    max_new_tokens=512,
                    temperature=0.1)

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-655b823e-68d7977e0e2ee0f10bbd4e66;8699087c-7880-422d-a882-d8451bab08d6)

Repository Not Found for url: https://huggingface.co/api/models/models/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [9]:
llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF",
                    model_file="llama-2-13b-ensemble-v5.Q5_K_M.gguf",
                    model_type="llama",
                    max_new_tokens=512,
                    temperature=0.1)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

In [ ]:
while True:
    chat_history = []
    #query = "What is the value of  GDP per capita of Finland provided in the data?"
    query = input(f"Input Prompt: ")
    if query == 'exit':
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = qa({"question":query, "chat_history":chat_history})
    print("Response: ", result['answer'])

Input Prompt:  What is the value of  GDP per capita of Finland provided in the data?


Response:   The value of GDP per capita of Finland provided in the data is 1.340.



Input Prompt:  Who is Kapil Dev


Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

Response:   Kapil Dev is a former Indian cricketer and a member of the Indian Cricket Hall of Fame. He was an all-rounder, playing as a right-arm fast-medium bowler and lower-order batsmanip middle order batsman in the-middle order right batman batman batman batting order batsidemiddlevehitting bats wicket batsmanaging batsman batman in the-middle order right-middle order right-middle order right order right order right order right order right order right order right order right order right order right order right order right order right-order bating middle order right-middle order right-middle order right-middle order right-middle order right-middle order right-middle order right-middle order right-middle order batsman batting order bats middle order bats middle order batsidemiddlevehitting bats right bat in the-order bats order right order right order right order bats right order bats right order bats right order right order right order right order right order right order right order